## Use python default cache feature to improve performance

In [3]:
# impport modules
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from functools import cache

In [4]:
# Create sample data: record the temperature of per hour for 5 cities in 10 days
np.random.seed(0)
date_range = pd.date_range(start="2024-04-01", end="2024-04-10", freq="h")
temp_df = pd.DataFrame(
    {
        "timestamp": np.tile(date_range, 5),
        "city": np.repeat(["beijing", "shanghai", "guangzhou", "shenzhen", "tianjin"], len(date_range)),
        "temperature": np.random.normal(loc=15, scale=10, size=(len(date_range) * 5,)),
    }
)
temp_df

,timestamp,city,temperature
0,2024-04-01 00:00:00,beijing,32.640523
1,2024-04-01 01:00:00,beijing,19.001572
2,2024-04-01 02:00:00,beijing,24.787380
3,2024-04-01 03:00:00,beijing,37.408932
4,2024-04-01 04:00:00,beijing,33.675580
...,...,...,...
1080,2024-04-09 20:00:00,tianjin,10.987022
1081,2024-04-09 21:00:00,tianjin,11.833447
1082,2024-04-09 22:00:00,tianjin,20.969065
1083,2024-04-09 23:00:00,tianjin,5.127133


In [5]:
# functions to caculate the average temperature for a given city
def compute_avg_daily_temp(date, city):
    daily_temp_df = temp_df[
        (temp_df["timestamp"].dt.date == pd.to_datetime(date).date())
        & (temp_df["city"] == city)
    ]
    print(daily_temp_df)
    return daily_temp_df["temperature"].mean()

@cache
def compute_avg_daily_temp_cached(date, city):
    daily_temp_df = temp_df[
        (temp_df["timestamp"].dt.date == pd.to_datetime(date).date())
        & (temp_df["city"] == city)
    ]
    return daily_temp_df["temperature"].mean()

In [12]:
compute_avg_daily_temp("2024-04-08", "tianjin")

17.723142045148002

In [13]:
compute_avg_daily_temp_cached("2024-04-08", "tianjin")

17.723142045148002

In [18]:
# let's use another day for testing the caching feature
%timeit -r 1 -n 1 compute_avg_daily_temp("2024-04-10", "tianjin")

6.06 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%timeit -r 1 -n 1 compute_avg_daily_temp_cached("2024-04-10", "tianjin")

1.27 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
%timeit -r 1 -n 1 compute_avg_daily_temp("2024-04-10", "tianjin")

1.93 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
%timeit -r 1 -n 1 compute_avg_daily_temp_cached("2024-04-10", "tianjin")

3.08 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Summary

cache实用场景：假设我们正在开发一个数据驱动的仪表板应用，缓存功能将推动以下实践。如果这个仪表板应用有多个用户，他们可能会查询相同的数据集并请求相同的汇总或计算指标。因此，缓存功能将帮助第二个及后续所有用户在更短的时间内得到结果。

在这个示例中，我们使用的每日数据是固定的。也就是说，一旦某天的平均温度被计算出来后，它就不会改变了。

### 其他考虑因素以及不应使用缓存的情况

并不是所有情况都推荐使用缓存。

1. 你的Python版本。@cache装饰器是在Python 3.9版本中引入的。如果你不能使用3.9或更高版本，请考虑使用@lru_cache，这是一个更全面的缓存功能。
2. 不要在非确定性函数中使用缓存。比如说，在函数中添加一个datetime.now()来获取当前的时间戳。获取当前时间戳的行为是非确定性的。相同的原则也适用于基于随机的函数。如果我们的随机函数每次生成相同的结果，那它就不再具有随机性啦！
3. 在具有“副作用”的函数中不应使用缓存。所谓“副作用”，我指的是除了返回值之外的操作，例如将文本写入文件或更新数据库表。如果我们对这些函数使用缓存，那么“副作用”将不会在第二次发生。换句话说，它只会在我们第一次调用函数时起作用。
4. 设备内存受限时，不要使用缓存。如果我们想缓存许多小结果，但担心结果集可能太多，那么使用 lru_cache 将是一个很不错的选择。

functools.lru_cache 是一个特别有用的装饰器，它提供了一个缓存机制，可以存储最近使用过的结果，并在达到缓存限制时自动丢弃最少使用的数据。这对于那些需要缓存大量但不希望无限增长内存使用的场景非常适合。

下面的例子展示了如何使用 lru_cache 来缓存一个计算函数的结果，该函数计算过去一定时间范围内的温度滚动平均值。在这个例子中，我们模拟缓存多个城市每小时的温度滚动平均值。

In [20]:
from functools import lru_cache
import random


# 模拟获取某城市某小时的温度
def get_temperature(city, hour):
    return random.uniform(20, 30)


# 使用lru_cache装饰器定义一个缓存温度平均值的函数
@lru_cache(maxsize=1000)  # 设定缓存大小为1000
def get_rolling_average(city, start_hour, end_hour):
    total_temp = 0
    count = end_hour - start_hour
    for hour in range(start_hour, end_hour):
        total_temp += get_temperature(city, hour)
    return total_temp / count

In [21]:
# 测试函数
# 获取并打印某城市连续多小时的温度平均值
city = "New York"
start_hour = 0
end_hour = 24
average_temp = get_rolling_average(city, start_hour, end_hour)
print(
    f"Average temperature from {start_hour} to {end_hour} in {city} is {average_temp:.2f}"
)

Average temperature from 0 to 24 in New York is 25.04


In [22]:
# 调用相同参数的函数多次以观察缓存效果
print(get_rolling_average(city, start_hour, end_hour))  # 直接从缓存中获取结果
print(
    get_rolling_average(city, start_hour + 1, end_hour + 1)
)  # 计算新的平均温度，并缓存结果

25.043627474356715
25.335899040311634


In [23]:
# 检查缓存的状态
print(get_rolling_average.cache_info())

CacheInfo(hits=1, misses=2, maxsize=1000, currsize=2)


这种方法非常适用于需要频繁重复计算相同或类似结果的情况，而 lru_cache 的限定大小则可以防止内存使用过多。这是一个高效利用计算资源的示例，尤其是在数据集较大时仍能保持性能。

5. 在数据涉及安全敏感的应用中不要使用缓存。请记住，缓存装饰不会加密其在内存中缓存的结果。这意味着当前操作系统中的其他进程可能会获取内存中缓存的信息。因此，请出于安全考虑，不要缓存任何敏感数据。

## 结论（Conclusion）

总的来说，我们在这篇文章中介绍了Python内置的 functools 模块中的 @cache 装饰器。它可以用来提高某些典型递归函数的性能，并且可以被认为是在我们的Python应用程序中实现缓存功能的最简单方法。

当然，它并不是万能的解决方案。我们应该意识到某些情况下绝对不应该使用它，例如在非确定性函数中。我们还需要注意内存使用，因为缓存的结果将被保留在内存中（使用 @lru_cache 作为替代）。